<a href="https://colab.research.google.com/github/sonu-coder/Spark-Playground/blob/main/SparkPlayground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [6]:
import findspark
findspark.init()

In [75]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import col,broadcast,rank,dense_rank,row_number,desc, monotonically_increasing_id as mid

In [8]:
spark = SparkSession.builder.getOrCreate()

In [26]:
df1 = spark.read.format('csv').option('header','true')\
                              .option('inferSchema','true')\
                              .load('/content/sample_data/california_housing_train.csv')
df2 = spark.read.csv(header=True,inferSchema=True,path='/content/sample_data/california_housing_test.csv')

In [39]:
df1 = df1.withColumn('Id',mid()+1)
df1 = df1.drop('longitude','latitude','total_rooms','population')
df2 = df2.select('longitude','latitude','total_rooms','population',(mid()+1).alias('Id'))

In [60]:
df_joined = df1.alias('df1').join(broadcast(df2.alias('df2')),df1.Id==df2.Id,'Inner').drop(df2.Id)

In [ ]:

df_joined = df_joined.select('Id',
                'housing_median_age',
                'total_bedrooms',
                'households',
                'median_income',
                'median_house_value',
                'longitude',
                'latitude',
                'total_rooms',
                'population')

In [86]:
window_spec = Window.orderBy(desc(col('population')))

In [87]:
df_window = df_joined.withColumn('rank',dense_rank().over(window_spec)).show()

+----+------------------+--------------+----------+-------------+------------------+---------+--------+-----------+----------+----+
|  Id|housing_median_age|total_bedrooms|households|median_income|median_house_value|longitude|latitude|total_rooms|population|rank|
+----+------------------+--------------+----------+-------------+------------------+---------+--------+-----------+----------+----+
| 979|              38.0|         206.0|     182.0|       1.8333|          127100.0|  -121.53|   38.48|    27870.0|   11935.0|   1|
|2187|              17.0|        1208.0|    1088.0|       2.4053|          105200.0|  -116.14|   34.45|     8796.0|   11139.0|   2|
|1147|              52.0|         389.0|     424.0|         3.79|          163400.0|  -117.27|   33.15|    23915.0|   10877.0|   3|
|2430|               4.0|         644.0|     597.0|       5.2497|          215000.0|   -117.2|   33.58|    30450.0|    9419.0|   4|
|1598|              16.0|         521.0|     472.0|       2.5625|          1